# Pristine GRAPE calculation of control fields for cnot implementation

Robert Johansson (robert@riken.jp)

In [152]:
%matplotlib inline
import matplotlib.pyplot as plt
import time
import numpy as np

In [153]:
from qutip import *
from qutip.control import *

In [154]:
T = 2 * np.pi 
times = np.linspace(0, T, 500)

In [155]:
T

6.283185307179586

"""
    Class for representing the result of a GRAPE simulation.

    Attributes
    ----------
    u : array
        GRAPE control pulse matrix.

    H_t : time-dependent Hamiltonian
        The time-dependent Hamiltonian that realize the GRAPE pulse sequence.

    U_f : Qobj
        The final unitary transformation that is realized by the evolution
        of the system with the GRAPE generated pulse sequences.
    
    """

In [156]:
U = sigmax()
R = 500
H_ops = [sigmay()]
H_labels = [r'$u_{y}$']

$U = \sigma_{x} $   
$H_{ops} = [\sigma_{y} ] $


In [157]:
omega_1 = 0.5
H0 =  omega_1 * sigmaz()

c_ops = []

# GRAPE

In [158]:
from qutip.control.grape import plot_grape_control_fields, _overlap, grape_unitary_adaptive, cy_grape_unitary

In [159]:
from scipy.interpolate import interp1d
from qutip.ui.progressbar import TextProgressBar

In [160]:
u0 = np.array([np.random.rand(len(times)) * 2 * np.pi * 0.05 for _ in range(len(H_ops))])

u0 = [np.convolve(np.ones(10)/10, u0[idx,:], mode='same') for idx in range(len(H_ops))]

u_limits = None #[0, 1 * 2 * pi]
alpha = None

In [ ]:
result = cy_grape_unitary(U, H0, H_ops, R, times, u_start=u0, u_limits=u_limits,
                          eps=2*np.pi*1, alpha=alpha, phase_sensitive=False,
                          progress_bar=TextProgressBar())

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/anaconda3/envs/qutip-env/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2963, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-161-1cb301d5355f>", line 3, in <module>
    progress_bar=TextProgressBar())
  File "/anaconda3/envs/qutip-env/lib/python3.6/site-packages/qutip/control/grape.py", line 387, in cy_grape_unitary
    for idx in range(M-1)]
  File "/anaconda3/envs/qutip-env/lib/python3.6/site-packages/qutip/control/grape.py", line 387, in <listcomp>
    for idx in range(M-1)]
  File "/anaconda3/envs/qutip-env/lib/python3.6/site-packages/qutip/control/grape.py", line 384, in _H_idx
    return H0 + sum([u[r, j, idx] * H_ops[j] for j in range(J)])
  File "/anaconda3/envs/qutip-env/lib/python3.6/site-packages/qutip/qobj.py", line 441, in __radd__
    return self + other
  File "/anaconda3/envs/qutip-env/lib/python3.6/site-packages/qutip/qobj.py", line 352, in __add__
    dat = other

KeyboardInterrupt: 

ERROR:tornado.general:Uncaught exception in ZMQStream callback
Traceback (most recent call last):
  File "/anaconda3/envs/qutip-env/lib/python3.6/site-packages/zmq/eventloop/zmqstream.py", line 432, in _run_callback
    callback(*args, **kwargs)
  File "/anaconda3/envs/qutip-env/lib/python3.6/site-packages/tornado/stack_context.py", line 276, in null_wrapper
    return fn(*args, **kwargs)
  File "/anaconda3/envs/qutip-env/lib/python3.6/site-packages/ipykernel/kernelbase.py", line 283, in dispatcher
    return self.dispatch_shell(stream, msg)
  File "/anaconda3/envs/qutip-env/lib/python3.6/site-packages/ipykernel/kernelbase.py", line 233, in dispatch_shell
    handler(stream, idents, msg)
  File "/anaconda3/envs/qutip-env/lib/python3.6/site-packages/ipykernel/kernelbase.py", line 421, in execute_request
    self._abort_queues()
  File "/anaconda3/envs/qutip-env/lib/python3.6/site-packages/ipykernel/kernelbase.py", line 636, in _abort_queues
    self._abort_queue(stream)
  File "/anacond

## Plot control fields for cnot gate in the presense of single-qubit tunnelling

plot_grape_control_fields(times, u, labels, uniform_axes=False):
    """
    Plot a series of plots showing the GRAPE control fields given in the
    given control pulse matrix u.

    Parameters
    ----------
    times : array
        Time coordinate array.

    u : array
        Control pulse matrix.

    labels : list
        List of labels for each control pulse sequence in the control pulse
        matrix.

    uniform_axes : bool
        Whether or not to plot all pulse sequences using the same y-axis scale.
    
    """

In [ ]:
plot_grape_control_fields(times,
                          result.u / (2 * np.pi), H_labels, uniform_axes=True);

## Fidelity/overlap

In [ ]:
U

In [ ]:
result.U_f

In [ ]:
result.U_f/1j

In [ ]:
hinton(result.U_f/1j)

In [ ]:
hinton(result.U_f)

In [ ]:
result.H_t[0]

In [ ]:
result.H_t[1]

In [ ]:
result.u

## Test numerical integration of GRAPE pulse

In [ ]:
U_f_numerical = propagator(
    result.H_t, times[-1], [], options=Odeoptions(nsteps=5000), args={})
U_f_numerical

In [ ]:
U_f_numerical / U_f_numerical[0,0]

In [ ]:
_overlap(result.U_f, U_f_numerical).real, abs(_overlap(result.U_f, U_f_numerical))**2

# Imports

Need to have jate.py in your folder

In [ ]:
%run jate.py #will import everything

## My code 

## Building parts

### Building the things to be calculated only once

In [ ]:
def maker(omega_1, H_0, H_1, T_s, Lin, d=2, gamma=0.1):
    r"""maker
    Makes all the things that remain constant throught the program, but are 
    repeatedly used.
    

    Parameters
    ----------
    omega_1 : float
              frequency corresponding to half of the difference between 
              energy levels of the qubit
              
    H_0     : Qobj
              Bare Hamiltonian 
              
    H_1     : Qobj
              Interaction Hamiltonian 
              
    T_s     : Qobj
              Unitary to be implemented in the Hilbert space
    
    Lin     : Qobj
              Linbladian operators

    d       : int
              Dimension of the matrix. Defaults to 2
    
    gamma   : float
              Damping constantof the Linbladian

    
    Returns
    -------
    
    ih0     : Qobj
              $I\otimes H_{0}$
              
    ih1     : Qobj
              $I\otimes H_{1}$

    h0ci    : Qobj
              $H_{0}^{*}\otimes I $

    h1ci    : Qobj
              $H_{1}^{*}\otimes I $

    T       : Qobj
              Target unitary transformed to the Liouville space

    linbladian : Qobj
                 The full lindbladian term as it appears on transformation to 
                 the Liouville space.
        
    """
    I = identity(d)
    L_I = tensor(I, I)
    ih0 = tensor(I, H_0) 
    ih1 = tensor(I, H_1) 
    h0ci = tensor(H_0.conj(), I) 
    h1ci = tensor(H_1.conj(), I)
    x_k = ih1 - h1ci
    term1 = tensor(Lin.trans(), Lin)
    term2 = tensor(I, ((Lin.dag())*(Lin)))
    term3 = tensor(((Lin.trans())*(Lin.conj())), I)
    lindbladian = 1j*(gamma)*(term1 - 0.5*(term2 + term3))
    T = tensor(T_s.trans(), T_s) # Transforming $T_{s}$ to liouville space
    
    
    return ih0, ih1, h0ci, h1ci, x_k, lindbladian, T, L_I

In [ ]:
omega_1 = 0.5
H_0 = omega_1*sigmaz() 
H_1 = sigmay()
T_s = sigmax() 
Lin = sigmaz()
ih0, ih1, h0ci, h1ci, x_k, lindbladian, T, L_I  = maker(omega_1,
                                                  H_0, H_1, T_s, 
                                                  Lin, d=2, gamma=0.0)

In [ ]:
L_I

### Building $A(t)$

In [ ]:
def A(xi):
    r"""making $A(t)$"""
    A = ih0 - h0ci + xi*(ih1 - h1ci) + lindbladian
    return A

In [ ]:
A(0.5)

### Building $L(t)$ and the Identity in the Liouville space

In [ ]:
def L(xi, dt):
    r"""Making $L(t) from $A(t)$"""
    L = (-1j*A(xi)*dt).expm()
    return L

In [ ]:
L(0.5, 0.001)

## Major functions

### Major functions 1

In [ ]:
# building the function to optimize (optimizee)
def L_vec(xi_vec, dt):
    r"""Building the vector of differential $L(t)$"""
    L_vec = [L(xi, dt) for xi in xi_vec] 
    return L_vec

In [ ]:
def fidelity_calc(A, B):
    r"""Making a generalised fidelity function"""
    first_part = (A - B).dag()
    second_part = (A - B)
    f_int = (first_part* second_part)
    f = f_int.tr()
    return f

In [ ]:
def L_full_maker(xi_vec, dt):
    r"""Building the $L(t)$ for the total time $t$"""
    xi_vec_size = xi_vec.size # finding the size of xi
    L_full = L_I # Identity for the for loop of L
    L_v = L_vec(xi_vec, dt) # calling L_vec
    for i in range(xi_vec_size): # generating L_full
        L_full = L_full*L_v[xi_vec_size - 1 - i]
    return L_full

In [ ]:
def F(xi_vec, dt):
    r"""Using the fidelity metric to find out the closeness between $T$
    and $L(t)$"""
    L_full = L_full_maker(xi_vec, dt)
    F = real(-fidelity_calc(T, L_full))   
    return F

### Testing major functions 1

In [ ]:
fidelity_calc(sigmax(), sigmay())

In [ ]:
fidelity_calc(sigmay(), sigmay())

In [ ]:
xi_vec_test = array([1.0, 2.0])
xi_vec_test

In [ ]:
xi_vec_test.size

In [ ]:
w_vec = [xi**2 for xi in xi_vec_test]
w_vec

In [ ]:
# F(xi_vec, dt)
F(xi_vec_test, 0.001)

In [ ]:
L_v = L_vec(xi_vec_test, 0.001)

In [ ]:
L_v

### Major Functions 2

In [ ]:
def L_comma_k_maker(xi_vec, k, dt):
    r"""Making of the derivative of full $L(t)$ at time $t_{k}$"""
    N = xi_vec.size 
    # Determining the size of xi, and thus the time_steps indirectly.
    L_v = L_vec(xi_vec, dt)# Making of the full $L(t)$
    inner_part = L_I # Beginner for the for loop
    for i in range(N):
        if i == ( N - 1 - k ):
            # The step at which $X_{k}(t)$ has to be inserted 
            inner_part = inner_part*x_k*L_v[k - 1]
        else:
            # Usual multiplications of $L_{k}$
            inner_part = inner_part*L_v[N - 1 - i]
    l_comma_k = inner_part
    return l_comma_k
    

In [ ]:
# L_comma_k_maker(xi_vec, k, dt)
L_comma_k_maker(xi_vec_test, 2, 0.001)

In [ ]:
def updater(xi_vec, dt, epsilon):
    r"""Implementing the GRAPE update step"""
    xi_vec_size = xi_vec.size # finding the size of xi
    L_full = L_full_maker(xi_vec, dt)
    di = []
    for k in range(xi_vec_size):
        # Building the thing to be added to the old function
        L_comma_k = L_comma_k_maker(xi_vec, k, dt)
        differentiated = T - L_comma_k
        plain = T - L_full
        c = -differentiated.dag()*plain
        d = -plain.dag()*differentiated
        inside = c.tr() + d.tr()
        di.append(epsilon*inside)

    diff = array(di)
    xi_new_vec = xi_vec + diff
    return diff, xi_new_vec
    

In [ ]:
#  updater(xi_vec, dt, epsilon)
updater(xi_vec_test, 0.001, 0.001)

In [ ]:
def terminator(max_iter, time_steps, total_time, epsilon):
    r"""Brief description of the function"""
    
    xi_initial =  1000*random_sample((time_steps,))
    dt = total_time/time_steps
    xi_diff, xi_new_vec = updater(xi_initial, dt, epsilon)
    
    for i in range(max_iter):
        if amax(xi_diff) < epsilon**2 :
            
            xi_final = xi_new_vec
            break
        else :
            xi_diff, xi_new_vec = updater(xi_new_vec, dt, epsilon)
            print(i)
            print(amax(xi_diff))
            
        
    xi_final = xi_new_vec    
    return xi_final

# Running stuff

### qutip grape

In [ ]:
xi_qutip = result.u

In [ ]:
len(xi_qutip)

In [ ]:
xi_qutip.shape

In [ ]:
xi_qutip.size

In [ ]:
len(xi_qutip[0])

In [ ]:
len(xi_qutip[1])

In [ ]:
len(xi_qutip[2])

In [ ]:
len(xi_qutip[0][0])

In [ ]:
len(xi_qutip[1][0])

In [ ]:
len(xi_qutip[1][0])

In [ ]:
len(xi_qutip[250][0])

In [ ]:
len(xi_qutip[250,0,:])

In [ ]:
len(xi_qutip[250,1,:])

In [ ]:
len(xi_qutip[-1,0,:])

In [ ]:
xi_qutip[0][0][1]

In [ ]:
xi_qutip

In [ ]:
xi_qutip[-1,0, :]

In [ ]:
#L_full_maker(xi_qutip, dt)
L_full_maker(xi_qutip[-1, 0, :], dt)

In [ ]:
2 * np.pi/500

In [ ]:
#L_full_maker(xi_qutip, dt)
blah = L_full_maker(xi_qutip[-1, 0, :], (2 * np.pi/500))
blah

In [ ]:
hinton(T)

In [ ]:
hinton(blah/blah[0,0])

In [ ]:
matrix_histogram(blah)

In [ ]:
matrix_histogram(T)

In [ ]:
T_s

In [ ]:
F(xi_qutip[-1, 0, :], dt)

In [ ]:
F(xi_qutip[-1, 0, :], (2*np.pi/500))

### Try1

In [ ]:
total_time = pi/omega_1
epsilon = 10**(-6)
max_iter = 10#10**4#1000#100#50#20
time_steps = 20
dt = total_time/time_steps

In [ ]:
hinton(T)

In [ ]:
xi_opt = terminator(max_iter, time_steps, total_time, epsilon)
xi_opt

In [ ]:
F(xi_opt, dt)

In [ ]:
max_iter

In [ ]:
T

In [ ]:
L_full_maker(xi_opt, dt)

In [ ]:
hinton(L_full_maker(xi_opt, dt))